In [ ]:
#| default_exp interactive

# Interactive

> Interactive components that require more elaborate implementations

In [ ]:
#| export
from fasthtml.common import *
from fhbasecoat.utils import *
from fhbasecoat.common import *
import fasthtml.components as fh
from fasthtml.jupyter import *
from fastcore.meta import delegates
from itertools import product
from enum import Enum, auto


In [ ]:
app = FastHTML(session_cookie="mysession")
rt = app.route

In [ ]:
# srv.stop()

In [ ]:
srv = JupyUvi(app)

In [ ]:
#| eval: false
from pathlib import Path
common_components = Path("../fhbasecoat/common.py").read_text()

When answering questions refer to already made common components with $`common_components`

#### Shared parts
Some of these interactive components have a few shared reusable components

In [ ]:
#| export
def Separator(role="separator", **kwargs):
    return Hr(role=role, **kwargs)

In [ ]:
def ItemHeader(*contents, **kwargs):
    return Div(*contents, role="heading", **kwargs)

### Dialog / Alert Dialog

In [ ]:
#| export
def DialogOpenButton(*args, did, cls=ButtonT.outline, **kwargs):
    return Button(*args, onclick=f"document.getElementById('{did}').showModal()", cls=cls, **kwargs)

In [ ]:
p(DialogOpenButton("Open", did="d1"))

In [ ]:
#| export
def DialogCloseButton(content=Icon("x"), cls="", **kwargs):
    return Button(content, onclick="this.closest('dialog').close()", cls=cls, aria_label="Close dialog", **kwargs)

Note: we're defaulting this to no button styling because it inherits a base style when inside the Dialog element.

In [ ]:
p(DialogCloseButton())

In [ ]:
#| export
def Dialog(*contents, id, title=None, desc=None, footer=None, close_btn=DialogCloseButton(), cls="w-full sm:max-w-[425px] max-h-[612px]", onclick="if (event.target === this) this.close()", **kwargs):
    parts = []
    if title or desc: parts.append(Header(H2(title), P(desc)))
    parts.append(Section(*contents))
    if footer: parts.append(Footer(footer))
    return fh.Dialog(Div(*parts, close_btn), id=id, cls=f"dialog {cls}", onclick=onclick, **kwargs)

In [ ]:
pw(
    DialogOpenButton("Edit Profile", did="dia1"),
    Dialog(
        Form(
            Div(
                Label("Name", fr="name"),
                Input(id="name", value="Pedro Duarte", autofocus=True, type="text"),
                cls="grid gap-3"
            ),
            Div(
                Label("Username", fr="username"),
                Input(id="username", value="@peduarte", type="text"),
                cls="grid gap-3"
            ),
            cls="grid gap-4"
        ),
        id="dia1",
        title="Edit profile",
        desc="Make changes to your profile here. Click save when you're done.",
        footer=Div(
            Button("Cancel", onclick="this.closest('dialog').close()", cls=ButtonT.outline),
            Button("Save changes", onclick="this.closest('dialog').close()"),
            cls="flex gap-2"
        )
    )
)

In [ ]:
#| export
@delegates(Dialog, keep=True)
def AlertDialog(*args, close_btn=None, onclick=None, **kwargs):
    return Dialog(*args, close_btn=close_btn, onclick=onclick, **kwargs)

Alert dialog is simply a dialog that requires you to click an action button to close (no 'x' button, and no background click close).

In [ ]:
pw(
    DialogOpenButton("Delete account", did="dia2"),
    AlertDialog(
        id="dia2",
        title="Are you absolutely sure?",
        desc="This action cannot be undone. This will permanently delete your account and remove your data from our servers.",
        footer=Div(
            Button("Cancel", onclick="this.closest('dialog').close()", cls=ButtonT.outline),
            Button("Save changes", onclick="this.closest('dialog').close()"),
            cls="flex gap-2"
        )
    )
)

### Dropdown menu

In [ ]:
#| export
def DropdownTriggerButton(*contents, did, cls=ButtonT.outline, **kwargs):
    return Button(*contents, id=f"{did}-trigger", aria_haspopup="menu", aria_controls=f"{did}-menu", aria_expanded="false", cls=cls, **kwargs)

In [ ]:
#| export
def DropdownItem(*contents, icon=None, shortcut=None, disabled=False, **kwargs):
    parts = []
    if icon: parts.append(icon)
    parts.extend(contents)
    if shortcut: parts.append(Span(shortcut, cls="text-muted-foreground ml-auto text-xs tracking-widest"))
    return Div(*parts, role="menuitem", aria_disabled="true" if disabled else None, **kwargs)

In [ ]:
#| export
def Dropdown(*contents, id, btn_content="Open", trigger_btn=None, cls="min-w-65", side="bottom", align="left"):
    """The dropdown menu requires a DropdownTriggerButton to activate. A default version is provided used btn_content value
    but can be overriden by passing a button to the trigger_btn param.
    """ 
    return Div(
        trigger_btn or DropdownTriggerButton(btn_content, did=id),
        Div(
            Div(
                *contents,
                role="menu", id=f"{id}-menu", aria_labelledby=f"{id}-trigger",
            ),
            id=f"{id}-popover", data_popover=True, aria_hidden="true", cls=cls,
            data_side=side, data_align=align,
        ),
        id=id, cls="dropdown-menu"
    )


In [ ]:
pw(
    Dropdown(
        ItemHeader("My Account", id="account-options"),
        DropdownItem("Profile", shortcut="⇧⌘P", icon=Icon("user")),
        DropdownItem("Billing", shortcut="⌘B", icon=Icon("credit-card")),
        Separator(),
        DropdownItem("Github"),
        DropdownItem("Support"),
        DropdownItem("API", disabled=True),
        Separator(),
        DropdownItem("Logout", shortcut="⇧⌘P"),
        id="testing", side="right", align="center",
    )
)

### Popover

Popover are similar to dropdowns but for richer inner content instead of a set of menu items.
This makes the implmentation a bit simpler, as you can put whatever you want inside the popover.

In [ ]:
def PopoverTriggerButton(*contents, pid, cls=ButtonT.outline, **kwargs):
    return Button(*contents, id=f"{pid}-trigger", aria_expanded="false", aria_controls=f"{pid}-popover", cls=cls, **kwargs)

def Popover(*contents, id, btn_content="Open popover", trigger_btn=None, cls="w-80", side="bottom", align="center"):
    """The popover requires a PopoverTriggerButton to activate. A default version is provided using btn_content value
    but can be overridden by passing a button to the trigger_btn param.
    """ 
    return Div(
        trigger_btn or PopoverTriggerButton(btn_content, pid=id),
        Div(
            *contents,
            id=f"{id}-popover", data_popover=True, aria_hidden="true", cls=cls,
            data_side=side, data_align=align,
        ),
        id=id, cls="popover"
    )

In [ ]:
pw(
    Popover(
        Div(
            Header(
                H4("Dimensions", cls="leading-none font-medium"),
                P("Set the dimensions for the layer.", cls="text-muted-foreground text-sm"),
                cls="grid gap-1.5"
            ),
            Form(
                Div(
                    Label("Width", fr="demo-popover-width"),
                    Input(type="text", id="demo-popover-width", value="100%", cls="col-span-2 h-8", autofocus=True),
                    cls="grid grid-cols-3 items-center gap-4"
                ),
                Div(
                    Label("Max. width", fr="demo-popover-max-width"),
                    Input(type="text", id="demo-popover-max-width", value="300px", cls="col-span-2 h-8"),
                    cls="grid grid-cols-3 items-center gap-4"
                ),
                Div(
                    Label("Height", fr="demo-popover-height"),
                    Input(type="text", id="demo-popover-height", value="25px", cls="col-span-2 h-8"),
                    cls="grid grid-cols-3 items-center gap-4"
                ),
                Div(
                    Label("Max. height", fr="demo-popover-max-height"),
                    Input(type="text", id="demo-popover-max-height", value="none", cls="col-span-2 h-8"),
                    cls="grid grid-cols-3 items-center gap-4"
                ),
                cls="form grid gap-2"
            ),
            cls="grid gap-4"
        ),
        id="demo-popover", align="left",
    )
)

### Select / Combobox

In [ ]:
#| export
def ListboxTriggerButton(
    icon=Icon("chevrons-up-down", cls="text-muted-foreground opacity-50 shrink-0"),
    cls=f"{ButtonT.outline} justify-between font-normal w-[180px]",
    **kwargs
):
    return Button(Span("", cls="truncate"), icon, type="button", aria_haspopup="listbox", aria_expanded="false", cls=cls, **kwargs)

In [ ]:
#| export
def SearchBar(value="", placeholder="Search items..."):
    return Header(
        Icon("search"),
        fh.Input(
            type="text", value=value, placeholder=placeholder,
            autocomplete="off", autocorrect="off", spellcheck="false",
            aria_autocomplete="list", role="combobox", 
            aria_expanded="false", 
        )
    )

In [ ]:
#| export
def ListBox(*contents, id, trigger_button=ListboxTriggerButton(), search_bar=SearchBar()):
    trigger_button.attrs.update(dict(aria_controls=f"{id}-listbox", id=f"{id}-trigger"))
    if search_bar: search_bar.attrs.update(dict(aria_controls=f"{id}-listbox", aria_labelledby=f"{id}-trigger"))
    return Div(
        trigger_button,
        Div(
            search_bar,
            Div(
                *contents,
                role="listbox", id=f"{id}-listbox", 
                aria_orientation="vertical", 
                aria_labelledby=f"{id}-trigger"
            ),
            id=f"{id}-popover", data_popover=True, aria_hidden="true"
        ),
        Input(type="hidden", name=f"{id}-value", value=""),
        id=id, cls="select"
    )

In [ ]:
#| export
def ListboxItem(content, value=None, selected=False, force=False, keywords=None, filter_text=None, disabled=False, **kwargs):
    attrs = {"role": "option", "data_value": value or content}
    if selected: attrs["aria_selected"] = "true"
    if force: attrs["data_force"] = "true"
    if keywords: attrs["data_keywords"] = keywords
    if filter_text: attrs["data_filter"] = filter_text
    if disabled: attrs["aria_disabled"] = "true"
    return Div(content, **attrs, **kwargs)

In [ ]:
#| exports
# Alias to make the items more discoverable
SelectItem = ComboboxItem = ListboxItem

In [ ]:
#| export
@delegates(ListBox, but=["search_bar"])
def Select(*args, searchable=False, **kwargs):
    search_bar = SearchBar() if searchable else None
    return ListBox(*args, search_bar=search_bar, **kwargs)

In [ ]:
#| export
@delegates(ListBox, but=["search_bar"])
def Combobox(*args, search_placeholder="Search items...", **kwargs):
    search_bar = SearchBar(placeholder=search_placeholder)
    return ListBox(*args, search_bar=search_bar, **kwargs)

Select example:

In [ ]:
pw(
    Select(
        ItemHeader("Fruits"),
        SelectItem("Apple"),
        SelectItem("Banana"),
        SelectItem("Blueberry"),
        id="fruit",
    )
)

Combobox example:

In [ ]:
pw(
    Combobox(
        ComboboxItem("Next.js"),
        ComboboxItem("SvelteKit"),
        ComboboxItem("Nuxt.js"),
        ComboboxItem("Remix"),
        ComboboxItem("Astro"),
        id="framework-select",
        search_placeholder="Search framework..."
    )
)

### Tabs

In [ ]:
#| export
def TabNav(nm, id, idx, selected=False):
    return fh.Button(nm, type="button", role="tab", id=f"{id}-tab-{idx}",  aria_controls=f"{id}-panel-{idx}", aria_selected="true" if selected else "false", tabindex="0")

def Tabs(contents:list, tablist:list, id:str, default_tab=0, orientation="horizontal", cls="w-96"):
    nav_items = [TabNav(o, id, idx, selected=(idx==default_tab)) for idx, o in enumerate(tablist)]
    for idx, content in enumerate(contents):
        active_dict = {"aria_selected": "true"} if idx==default_tab else {"aria_selected": "false", "hidden": True}
        content.attrs.update({"role": "tabpanel", "id": f"{id}-panel-{idx}", "aria_labelledby": f"{id}-tab-{idx}", "tabindex": "-1", **active_dict})

    return Div(
        Nav(
            *nav_items,
             role="tablist", aria_orientation=orientation, cls="w-full",
        ),
        *contents,
        cls=f"tabs {cls}", id=id,
    )

In [ ]:
pw(
    Tabs(
        contents=[
            Div(
                Card(
                    Form(
                        Div(
                            Label("Name", fr="demo-tabs-account-name"),
                            Input(type="text", id="demo-tabs-account-name", value="Pedro Duarte"),
                            cls="grid gap-3"
                        ),
                        Div(
                            Label("Username", fr="demo-tabs-account-username"),
                            Input(type="text", id="demo-tabs-account-username", value="@peduarte"),
                            cls="grid gap-3"
                        ),
                        cls="form grid gap-6"
                    ),
                    title="Account",
                    desc="Make changes to your account here. Click save when you're done.",
                    footer=Button("Save changes", type="button", cls="btn"),
                    cls="card",
                ),
            ),
            Div(
                Card(
                    Form(
                        Div(
                            Label("Current password", fr="demo-tabs-password-current"),
                            Input(type="password", id="demo-tabs-password-current"),
                            cls="grid gap-3"
                        ),
                        Div(
                            Label("New password", fr="demo-tabs-password-new"),
                            Input(type="password", id="demo-tabs-password-new"),
                            cls="grid gap-3"
                        ),
                        cls="form grid gap-6"
                    ),
                    title="Password",
                    desc="Change your password here. After saving, you'll be logged out.",
                    footer=Button("Save Password", type="button", cls="btn"),
                    cls="card",
                ),
            )
        ],
        tablist=["Account", "Password"],
        id="demo-tabs-with-panels",
    )
)